# 🚀 Llama-3-8B Kurumsal Chatbot Fine-Tuning
## Optuna ile Hiperparametre Optimizasyonu

Bu notebook, kurumsal chatbot için Llama-3-8B modelini fine-tune eder ve Optuna ile en iyi hiperparametreleri bulur.

### Özellikler:
- ✅ Optuna ile otomatik hiperparametre optimizasyonu
- ✅ Perplexity, Cross-Entropy Loss, F1 Score metrikleri
- ✅ A/B test desteği
- ✅ JSON formatında sonuç çıktısı
- ✅ Colab uyumlu


## 1. Gerekli Kütüphaneleri Yükle


In [ ]:
# GPU kontrolü
import torch
print(f"CUDA Kullanılabilir: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Versiyonu: {torch.version.cuda}")


In [ ]:
# Gerekli paketleri yükle
%pip install -q transformers accelerate datasets optuna scikit-learn pyyaml sentencepiece numpy pandas


In [ ]:
# HuggingFace token ayarla (gerekirse)
# from huggingface_hub import login
# login()  # Token'ınızı buraya girin


## 2. Dataset Hazırlama


In [ ]:
# Dataset dosyanızı yükleyin
# Örnek: Google Drive'dan veya doğrudan yükleyin

import json
import os

# Dataset dizinini oluştur
os.makedirs("data", exist_ok=True)

# Örnek dataset oluşturma (kendi dataset'inizi kullanın)
sample_data = [
    {"text": "Müşteri: Merhaba, ürününüz hakkında bilgi alabilir miyim? Asistan: Tabii ki! Size nasıl yardımcı olabilirim?"},
    {"text": "Müşteri: Fiyat bilgisi almak istiyorum. Asistan: Hangi ürün için fiyat bilgisi istiyorsunuz?"},
    # ... daha fazla örnek ekleyin
]

# Dataset'i JSONL formatında kaydet
with open("data/train.jsonl", "w", encoding="utf-8") as f:
    for item in sample_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ {len(sample_data)} örnek dataset'e eklendi")


## 3. Optuna Optimizasyon Scriptini Yükle ve Çalıştır


In [ ]:
# optuna_optimization.py dosyasını buraya yükleyin
# Alternatif olarak, script'i doğrudan çalıştırabilirsiniz

# Script'i çalıştır
!python optuna_optimization.py \
    --dataset data/train.jsonl \
    --n_trials 10 \
    --output optuna_results.json \
    --study_name llama3_8b_chatbot_optimization


## 4. Sonuçları Görüntüle


In [ ]:
import json
import pandas as pd

# JSON sonuçlarını yükle
with open("optuna_results.json", "r", encoding="utf-8") as f:
    results = json.load(f)

print("🏆 En İyi Trial:")
print(f"Trial Numarası: {results['best_trial']['number']}")
print(f"Objective Değeri: {results['best_trial']['value']:.4f}")
print("\n🔧 En İyi Parametreler:")
for key, value in results['best_trial']['params'].items():
    print(f"  {key}: {value}")

if 'best_metrics' in results and results['best_metrics']:
    print("\n📏 En İyi Metrikler:")
    metrics = results['best_metrics']
    print(f"  Perplexity: {metrics.get('perplexity', 'N/A'):.2f}")
    print(f"  Cross-Entropy Loss: {metrics.get('cross_entropy_loss', 'N/A'):.4f}")
    print(f"  F1 Score: {metrics.get('f1_score', 'N/A'):.4f}")
    print(f"  Accuracy: {metrics.get('accuracy', 'N/A'):.4f}")


In [ ]:
# Tüm trial'ları DataFrame olarak göster
trials_data = []
for trial in results['all_trials']:
    row = trial['params'].copy()
    row['trial_number'] = trial['number']
    row['objective_value'] = trial['value']
    if 'metrics' in trial:
        row['perplexity'] = trial['metrics'].get('perplexity', None)
        row['f1_score'] = trial['metrics'].get('f1_score', None)
        row['cross_entropy_loss'] = trial['metrics'].get('cross_entropy_loss', None)
    trials_data.append(row)

df = pd.DataFrame(trials_data)
print("📊 Tüm Trial'lar:")
display(df)
